In [ ]:
import pandas as pd
import geopandas as gpd
import folium
import dataset
from shapely.geometry import LineString, Point, MultiPoint

In [ ]:
%%time

db = dataset.connect('sqlite:///stravaweb/strava_activities.sqlite')
BASE_POINT = Point(44.01016, 56.31832)
SIMPLIFY_THRESHOLD = 0.0001
MAX_DISTANCE = 5  # in degrees
crs = {'init': 'epsg:3857'}

df = pd.DataFrame(x for x in db['activities'].all() if x['latlng'])
df['dateteime'] = pd.to_datetime(df['datetime'])
df = df.drop(columns=['time', 'velocity_smooth', 'id'])
gdf = gpd.GeoDataFrame(df.drop(columns=['latlng']), geometry=[LineString([(pt[1], pt[0]) for pt in x]) for x in df.latlng], crs=crs)
gdf['distance_to_base'] = gdf.geometry.distance(BASE_POINT)
gdf['start_loc'] = gpd.GeoSeries([Point((x[0][1], x[0][0])) for x in df.latlng], crs=crs)
print(gdf.shape[0])
gdf.dtypes

In [ ]:
gdf.distance_to_base.hist(bins=10)
gdf.start_loc.plot()

In [ ]:
gdf[gdf.distance_to_base > MAX_DISTANCE].shape[0]

In [ ]:
%%time
import bisect
from shapely.geometry.collection import GeometryCollection


def highlight_function(feature):
    return {
        'fillColor': 'green',
        'color': 'green',
        'weight': 3,
        'dashArray': '5, 5'
    }


# FeatureCollection is used here because geometry collection has bug with highlighting
class FeatureCollection(GeometryCollection):
    @property
    def __geo_interface__(self):
        features = []
        for geom in self.geoms:
            features.append({'type': 'Feature', 'geometry': geom.__geo_interface__})
        return dict(type='FeatureCollection', features=features)


def generate_map(selected_df):
    minmax = selected_df.agg({'datetime': ['min', 'max']}).values
    min_date, max_date = pd.to_datetime(minmax[0][0]), pd.to_datetime(minmax[1][0])
    n_groups = 10
    delta = ((max_date - min_date) // n_groups)
    groups = [(min_date + delta * i).date() for i in range(1, n_groups + 1)]

    groups_by_key = {}

    my_map = folium.Map(location=[BASE_POINT.y, BASE_POINT.x], zoom_start=12)
    for row in selected_df.sort_values('datetime').itertuples():
        tooltip = '{} - {}'.format(row.athlete_name, row.title)
        route = row.geometry.simplify(SIMPLIFY_THRESHOLD, preserve_topology=False)
        date = pd.to_datetime(row.datetime).date()
        date_from_group = groups[bisect.bisect_left(groups, date)]
        start = date_from_group - delta
        layer_key = '{} - {}'.format(start.isoformat(), date_from_group.isoformat())

        fg = groups_by_key.get(layer_key)
        if not fg:
            fg = folium.FeatureGroup(layer_key).add_to(my_map)
            groups_by_key[layer_key] = fg

        geo_obj = folium.GeoJson(
            FeatureCollection([row.start_loc, route]), tooltip=tooltip, 
            highlight_function=highlight_function,
        ).add_to(fg)
        popup = '<a href="https://www.strava.com/athletes/{athlete_id}" target="_blank">{athlete_name}</a> - <a href="https://www.strava.com/activities/{activity_id}" target="_blank">{activity_name} ({datetime})</a>'.format(
            athlete_id=row.athlete_id, athlete_name=row.athlete_name, activity_id=row.activity_id, activity_name=row.title, datetime=row.datetime
        )
        folium.Popup(popup).add_to(geo_obj)

    folium.LayerControl(sortLayers=True).add_to(my_map)
    return my_map

my_map = generate_map(gdf[gdf.distance_to_base < MAX_DISTANCE])
my_map

In [ ]:
my_map.save('result.html')

In [ ]:
sel = gdf[gdf.athlete_name.str.contains('Haritonov')].iloc[0]
display(GeometryCollection([sel.start_loc, sel.geometry.centroid, sel.geometry.representative_point(), sel.geometry, sel.geometry.convex_hull]))

In [ ]:
from shapely.strtree import STRtree
tree = STRtree(gdf.geometry)

In [ ]:
%%time

len(tree.query(BASE_POINT.buffer(0.0000000001)))

In [ ]:
for i in range(10):
    display(GeometryCollection([gdf.geometry[i].centroid, gdf.geometry[i].representative_point(), gdf.geometry[i]]))